# Testing impact of stricter triggers

In [1]:
import copy
import plotly
import numpy as np
import pandas as pd

from plotly import tools
import plotly.tools as tls 
import plotly.plotly as py  
import plotly.graph_objs as go
from plotly.graph_objs import *
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
%matplotlib inline

import terminal_optimization.visualisation               as visualisation
import terminal_optimization.forecast                    as forecast
import terminal_optimization.investment_decisions        as invest
import terminal_optimization.infrastructure              as infra
import terminal_optimization.financial_analysis          as financial
import terminal_optimization.initial_terminal            as initial
import terminal_optimization.investment_decisions_future as future_invest

# Log in to Plotly servers
#plotly.tools.set_credentials_file(username='jorisneuman', api_key='zButeTrlr5xVETcyvazd')
#plotly.tools.set_credentials_file(username='wijzermans', api_key='FKGDvSah3z5WCNREBZEq')
plotly.tools.set_credentials_file(username='wijnandijzermans', api_key='xeDEwwpCK3aLLR4TIrM9')

# Applied colours within the plots
colors = ['rgba(55, 83, 109, 1)', 'rgba(55, 83, 109, 0.8)', 'rgba(55, 83, 109, 0.5)',
          'rgb(129, 180, 179, 1)', 'rgb(79, 127, 127, 1)', 'rgb(110,110,110)'] 

## 1 Traffic Generator

In [2]:
def traffic_generator(nr_generations, start_year, simulation_window):
    
    ###################################################################################################
    # Traffic projections on which estimate is based
    ###################################################################################################

    # Import commodities from package
    maize   = forecast.bulk_commodities(**forecast.maize_data)
    soybean = forecast.bulk_commodities(**forecast.maize_data)
    wheat   = forecast.bulk_commodities(**forecast.wheat_data)

    # Maize - demand
    maize.historic = [400000,450000,500000,600000,550000]
    maize.rate     = 1.0636 
    maize.sigma    = 0.0374

    # If defined number of traffic generations is even, one iteration is added in order to be able to identify the median
    if nr_generations % 2 == 0:
        nr_generations = nr_generations + 1 

    # Create traffic projections
    traffic_projections = []
    for iterations in range(nr_generations):
        maize.random_scenario(start_year, simulation_window, maize.historic, maize.rate, maize.sigma)
        traffic_projections.append(maize.demand)
    
    # Calculate the median traffic projection (i.e. traffic scenario)
    traffic_scenario = []
    
    for i in range (len(maize.historic)):
        traffic_scenario.append(
            maize.historic[i])
        
    for year in range (len(maize.historic), len(maize.historic) + simulation_window):
        throughputs = []
        for i in range (len(traffic_projections)):
            throughputs.append(
                traffic_projections[i][year])
        traffic_scenario.append(
            np.median(throughputs))
        
    # Define commodity demand     
    maize.demand   = traffic_scenario 
    soybean.demand = [0]*len(maize.demand)
    wheat.demand   = [0]*len(maize.demand)
    commodities    = [maize, soybean, wheat]
    
    # Import vessels from package
    handysize = forecast.vessel(**forecast.handysize_data)
    handymax  = forecast.vessel(**forecast.handymax_data)
    panamax   = forecast.vessel(**forecast.panamax_data)
    vessels = [handysize, handymax, panamax]
    
    # Calculate yearly calls
    vessels = forecast.vessel_call_calc(vessels, commodities, simulation_window)

    # Plot forecast
    fig = visualisation.scenario(traffic_projections, commodities)
    
    return vessels, commodities, fig, traffic_projections, traffic_scenario

## 2 Terminal Design

In [3]:
def design(chosen_method, terminal, vessels, commodities, start_year, simulation_window):
    
    for i in range (start_year, start_year + simulation_window): 
        
        year = i 
        timestep = year - start_year

        ##################################################################
        # Investment Decisions (current performance method)
        ##################################################################     
        
        if chosen_method == 'Current performance method':
            
            # Berths and cranes
            terminal.berths, terminal.cranes = invest.berth_invest_decision(terminal.berths, terminal.cranes, commodities, vessels, terminal.allowable_vessel_waiting_time, year, timestep, operational_hours)

            # Quay
            terminal.quays = invest.quay_invest_decision(terminal.quays, terminal.berths, vessels, year, timestep)

            # Storage
            storage_type = 'Silos'
            terminal.storage = invest.storage_invest_decision(terminal.storage, terminal.dwell_time, vessels, storage_type, commodities, year, timestep, operational_hours)

            # Loading stations
            terminal.stations, terminal.trains = invest.station_invest_decision(terminal.stations, forecast.train(**forecast.train_data), terminal.allowable_train_waiting_time, commodities, timestep, year, operational_hours)

            # Conveyors
            terminal.quay_conveyors = invest.quay_conveyor_invest_decision(terminal.quay_conveyors, terminal.berths, terminal.cranes, commodities, year, timestep, operational_hours)
            terminal.hinterland_conveyors = invest.hinterland_conveyor_invest_decision(terminal.hinterland_conveyors, terminal.stations, commodities,  year, timestep, operational_hours)

        ##################################################################
        # Investment Decisions (Perfect foresight and forecast method)
        ################################################################## 
        
        if chosen_method == 'Perfect foresight method' or chosen_method == 'Forecast based method':
            
            # Create forecast and accompanying vessel calcs
            commodities = forecast.forecaster(chosen_method, 'Linear', commodities, forecast_window, hindcast_window, timestep)
            vessels = forecast.forecast_call_calc(vessels, commodities, simulation_window)
            
            # Berths and cranes
            terminal.berths, terminal.cranes = future_invest.berth_invest_decision(terminal.berths, terminal.cranes, commodities, vessels, terminal.allowable_vessel_waiting_time, year, timestep, operational_hours)

            # Quay
            terminal.quays = future_invest.quay_invest_decision(terminal.quays, terminal.berths, vessels, year, timestep)

            # Storage
            storage_type = 'Silos'
            terminal.storage = future_invest.storage_invest_decision(terminal.storage, terminal.dwell_time, vessels, storage_type, commodities, year, timestep, operational_hours)

            # Loading stations
            terminal.stations, terminal.trains = future_invest.station_invest_decision(terminal.stations, forecast.train(**forecast.train_data), terminal.allowable_train_waiting_time, commodities, timestep, year, operational_hours)

            # Conveyors
            terminal.quay_conveyors = future_invest.quay_conveyor_invest_decision(terminal.quay_conveyors, terminal.berths, terminal.cranes, commodities, year, timestep, operational_hours)
            terminal.hinterland_conveyors = future_invest.hinterland_conveyor_invest_decision(terminal.hinterland_conveyors, terminal.stations, commodities, year, timestep, operational_hours)      
        
        ##################################################################
        # Business logic
        ################################################################## 
        
        # Terminal throughput
        terminal = financial.throughput_calc(terminal, commodities, vessels, terminal.trains, operational_hours, timestep, year)
        # Revenue
        terminal.revenues = financial.revenue_calc(terminal.revenues, terminal.throughputs, commodities, year, timestep)       
        # Capex
        terminal.capex = financial.capex_calc(terminal, year, timestep)
        # Labour costs
        terminal.labour = financial.labour_calc(terminal, year, timestep, operational_hours)
        # Maintenance costs
        terminal.maintenance = financial.maintenance_calc(terminal, year, timestep)
        # Energy costs
        terminal.energy = financial.energy_calc(terminal, year, operational_hours, timestep)
        # Insurance costs
        terminal.insurance = financial.insurance_calc(terminal, year, timestep)
        # Lease costs 
        terminal.lease = financial.lease_calc(terminal, year,timestep)
        # Demurrage costs
        terminal.demurrage = financial.demurrage_calc(terminal.demurrage, terminal.berths, terminal.cranes, commodities, vessels, operational_hours, timestep, year)
        # Residual value calculations 
        terminal.residuals = financial.residual_calc(terminal, year, timestep)
        # Profits
        terminal.profits = financial.profit_calc(terminal, simulation_window, timestep, year, start_year)
        # Opex
        terminal.opex = financial.opex_calc(terminal, year, timestep)  
        
    #WACC depreciated profits
    terminal.WACC_cashflows = financial.WACC_calc(terminal.project_WACC, terminal.profits, terminal.revenues, terminal.capex, terminal.opex, simulation_window, start_year)
    
    # Combine all cashflows
    terminal.cashflows = financial.cashflow_calc(terminal, simulation_window, start_year) 
    
    #NPV 
    terminal.NPV = financial.NPV_calc(terminal.WACC_cashflows)
        
    return terminal

# 3 Simulation

## 3.1 Two crane capex scenarios
Scenario 1: < 20% waiting time<br>
Scenario 2: < 40% waiting time

In [8]:
# Simulation parameters
start_year        = 2018   # start year of simulation
simulation_window = 20     # forecast 20 years
end_year          = start_year + simulation_window - 1
operational_hours = 5840   # operational hours per year (16 hours per day 365 days a year)
project_WACC      = 0.1114 # Applied project weighted average cost of capital
nr_generations    = 11   # Number of traffic projections generated

chosen_method = 'Current performance method'
scenarios = []
cranes_online = []

# Make traffic projections
vessels, commodities, fig, traffic_projections, traffic_scenario = traffic_generator(nr_generations, start_year, simulation_window)
maize = commodities[0]
demand = [700000, 700000, 700000, 700000, 700000]
for i in range(20):
    demand.append(
        700000+150000*i)
maize.demand = demand

# Scenario 1 - 20% acceptable vessel waiting time
# Performance triggers (static values)
triggers = []
triggers.append([0.20, 'Acceptable waiting vessel time as factor of service time'])
triggers.append([18  , 'Dwell time of cargo in storage, presented in days'])         # PIANC guidelines
triggers.append([0.50, 'Acceptable waiting train time as factor of service time'])

terminal = initial.terminal(project_WACC, triggers)
terminal = design(chosen_method, terminal, vessels, commodities, start_year, simulation_window)

crane_capex = []
years = []
for i in range(len(terminal.capex)):
    years.append(
        terminal.capex[i].year)
    crane_capex.append(
        terminal.capex[i].cranes)
scenarios.append(
        crane_capex)

n_cranes = []
for year in years:
    # Number of cranes online
    online = []
    for i in range(4):
        for j in range(len(terminal.cranes[i])):
            if year >= terminal.cranes[i][j].online_date:
                online.append(1)
    n_cranes.append(np.sum(online))
cranes_online.append(
    n_cranes)

# Scenario 2 - 40% acceptable vessel waiting time
# Performance triggers (static values)
triggers = []
triggers.append([0.40, 'Acceptable waiting vessel time as factor of service time'])
triggers.append([18  , 'Dwell time of cargo in storage, presented in days'])         # PIANC guidelines
triggers.append([0.50, 'Acceptable waiting train time as factor of service time'])

terminal = initial.terminal(project_WACC, triggers)
terminal = design(chosen_method, terminal, vessels, commodities, start_year, simulation_window)

crane_capex = []
years = []
for i in range(len(terminal.capex)):
    years.append(
        terminal.capex[i].year)
    crane_capex.append(
        terminal.capex[i].cranes)
scenarios.append(
        crane_capex)

n_cranes = []
for year in years:
    # Number of cranes online
    online = []
    for i in range(4):
        for j in range(len(terminal.cranes[i])):
            if year >= terminal.cranes[i][j].online_date:
                online.append(1)
    n_cranes.append(np.sum(online))
cranes_online.append(
    n_cranes)

## 3.2 Visualize Financial Impact

In [10]:
trace1 = go.Scatter(
    name='Traffic volume',
    x=maize.years[5:],
    y=maize.demand[5:],
    mode='lines',
    line=dict(
        color='rgb(214, 39, 40)', 
        width=3,
    )
)

trace2 = {
    'x': maize.years[5:],
    'y': cranes_online[0],
    'name': 'Number of cranes',
    'type': 'bar',
    'showlegend': False,
    'marker': dict(color=colors[0])
}

trace3 = {
    'x': maize.years[5:],
    'y': scenarios[0],
    'name': 'Crane capex',
    'type': 'bar',
    'showlegend': False,
    'marker': dict(color='rgb(214, 39, 40)')
}

trace4 = {
    'x': maize.years[5:],
    'y': cranes_online[1],
    'name': 'Number of cranes',
    'type': 'bar',
    'marker': dict(color=colors[0])
}

trace5 = {
    'x': maize.years[5:],
    'y': scenarios[1],
    'name': 'Crane capex',
    'type': 'bar',
    'marker': dict(color='rgb(214, 39, 40)')
}

fig = tools.make_subplots(rows=3, cols=2,
                          specs=[[{'rowspan': 1, 'colspan': 2}, None],
                                 [{}, {}],
                                 [{}, {}]],
                          print_grid=True)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 2, 1)
fig.append_trace(trace3, 3, 1)
fig.append_trace(trace4, 2, 2)
fig.append_trace(trace5, 3, 2)


fig['layout'].update(
    height=800, 
    width=900, 
    title='The financial impact of stricter performance triggers',
    xaxis = dict(
        range=[2020,2037]),
    yaxis = dict(
        range=[10000,3600000],
        title='Traffic volume (t/year)',
        ticksuffix = 't'),
    yaxis2 = dict(
        range=[0,6.1],
        title='Number of cranes'),
    yaxis3 = dict(
        range=[0,6.1]),
    yaxis4 = dict(
        range=[-12100000, 0],
        title='Acquisition costs',
        tickprefix='$'),
    yaxis5 = dict(
        range=[-12100000, 0],
        tickprefix='$'),
    annotations=[dict(
                    x=1,
                    y=-0.20,
                    showarrow=False,
                    text='Source: URL or QR Code',
                    xref='paper',
                    yref='paper'),
                dict(
                    x=0.12,
                    y=-0.12,
                    showarrow=False,
                    text='Allowable vessel waiting time:<br>20% of service time',
                    xref='paper',
                    yref='paper'),
                dict(
                    x=0.92,
                    y=-0.12,
                    showarrow=False,
                    text='Allowable vessel waiting time:<br>40% of service time',
                    xref='paper',
                    yref='paper')],
    margin=dict(
            l=90,
            r=50,
            b=150,
            t=70)
    
)

py.iplot(fig, filename='stacked-subplots')

This is the format of your plot grid:
[ (1,1) x1,y1           -      ]
[ (2,1) x2,y2 ]  [ (2,2) x3,y3 ]
[ (3,1) x4,y4 ]  [ (3,2) x5,y5 ]

